# Estimate Roughness from LCDB v5

In [9]:
import rioxarray
import pathlib
import geopandas
import shapely
import geoapis.vector
import dotenv
import os

In [13]:
lcdb_id = 104400

In [4]:
base_path = pathlib.Path(r"C:\Local\data\catchments\Waikanae\caches\roughness\land_cover")

In [3]:
x0=1767800
x1=1775500
y0=5469612
y1=5477770
crs = 2193
bbox = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x0,y1],[x1,y1],[x1,y0]])], crs=crs)

In [21]:
# Lookup table values derived from - https://apps.dtic.mil/sti/tr/pdf/ADA274550.pdf
# And also some set from values given from Graeme - will need to replace with something to cite
ZO_LOOKUP = {
    "Transport Infrastructure": 0.00002, # page 30 Blacktop or concrete 
    'Exotic Forest': 0.4, # Page Fairly level wooded country 
    'Built-up Area (settlement)': 0.4,  # Page 30 Village
    'Low Producing Grassland': 0.0075, # page 25 Thick grass, 5 to 6 cm high  
    'River': 0.004, # Value picked by Rose & Alice. To replace from table 
    'Herbaceous Freshwater Vegetation': 0.04, # Value picked by Rose & Alice. To replace from table 
    'Indigenous Forest': 0.4, # Page 26 Fairly level wooded country 
    'Broadleaved Indigenous Hardwoods': 0.4, # Page 26 Fairly level wooded country 
    'Lake or Pond': 0.004,  # Value picked by Rose & Alice. To replace from table 
    'Manuka and/or Kanuka': 0.4, # Page 26 Fairly level wooded country 
    'Gorse and/or Broom': 0.25, # Page 26 Brush, scrub growth, dense 
    'High Producing Exotic Grassland': 0.09, # page 26 Thick grass, 50 cm high 
    'Deciduous Hardwoods': 0.4, # Page 26 Fairly level wooded country 
    'Sand or Gravel': 0.008,  # Value picked by Rose & Alice. To replace from table 
    'Mixed Exotic Shrubland': 0.25, # Page 26 Brush, scrub growth, dense 
    'Surface Mine or Dump': 0.4,  # Value picked by Rose & Alice. To replace from table 
    'Orchard, Vineyard or Other Perennial Crop': 0.31, # Page 28 Citrus orchard 3.2 m 
    'Forest - Harvested': 0.4, # Page 26 Forest clearings, cutover areas 
    'Gravel or Rock': 0.01,  # Value picked by Rose & Alice. To replace from table 
    'Fernland': 0.25, # Page 26 Brush, scrub growth, dense  
    "Matagouri or Grey Scrub": 0.16, # Page 26 Brush, scrub growth, open 
}

In [16]:
if not (base_path / "lcdb_v5.gpkg").exists():
    dotenv.load_dotenv()
    lris_key = os.environ.get("LRIS_API", None)
    fetcher = geoapis.vector.Lris(key=lris_key,
                                  bounding_polygon=bbox,
                                  verbose=True,
                                 )
    vector = fetcher.run(layer=lcdb_id)
    vector.to_file(base_path / "lcdb_v5.gpkg")
else:
    vector = geopandas.read_file(base_path / "lcdb_v5.gpkg")

# Clean data
vector = vector.drop(columns=['Name_2012', 'Name_2008', 'Name_2001', 'Name_1996',
       'Class_2012', 'Class_2008', 'Class_2001', 'Class_1996',
       'Wetland_12', 'Wetland_08', 'Wetland_01', 'Wetland_96',
       'Onshore_12', 'Onshore_08', 'Onshore_01', 'Onshore_96',
       'EditAuthor', 'EditDate'])

2e-05